<a href="https://colab.research.google.com/github/muraleee/collab-stuff/blob/main/sample.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip install -qU huggingface_hub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.0/521.0 kB 9.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
transformers 4.57.3 requires huggingface-hub<1.0,>=0.34.0, but you have huggingface-hub 1.2.3 which is incompatible.


In [ ]:
%pip install huggingface_hub


In [8]:
import os
import getpass

os.environ["HF_TOKEN"] = getpass.getpass("Hugging Face Token:")

Hugging Face Token:··········


In [9]:
HF_TOKEN = os.environ["HF_TOKEN"]

In [10]:
from huggingface_hub import InferenceClient

client = InferenceClient(api_key=HF_TOKEN, model="moonshotai/Kimi-K2-Thinking")

In [14]:
response = client.chat.completions.create(
    messages=[
        {"role" : "user", "content":"who are you?"}
        ]
)

In [22]:
print(response.choices[0].message.__dict__)

{'role': 'assistant', 'content': "I am Kimi, an AI assistant created by Moonshot AI. I'm designed to handle complex tasks and can process very long conversations or documents.", 'reasoning': None, 'tool_call_id': None, 'tool_calls': None, 'reasoning_content': 'The user asks "who are you?" This is a straightforward identity question. I need to answer truthfully and concisely based on the provided identity card.\n\nKey points to cover:\n- I am an AI assistant.\n- My name is Kimi.\n- I was created by Moonshot AI (月之暗面科技有限公司).\n- I should highlight a key capability or feature that defines me, such as my long context window or MoE architecture, but in a natural, non-technical way if possible.\n- The tone should be helpful, direct, and aligned with the identity card.\n\nStructure:\n1. Directly state my identity: name, nature (AI assistant), and creator.\n2. Add a brief, distinguishing feature to make it memorable and accurate to the card.\n3. Keep it concise and natural.\n\nDrafting:\n"I am 

In [23]:
def get_temperature(city: str):
    """
    Get the current weather in a given city.
    """
    if city.lower() == "san francisco":
        return "72"
    if city.lower() == "paris":
        return "75"
    if city.lower() == "tokyo":
        return "73"
    return "70"


In [24]:
get_temperature_tool_schema = {
    "type": "function",
    "function": {
        "name": "get_temperature",
        "description": "Get the current temperature in a given city.",
        "parameters": {
            "type": "object",
            "properties": {
                "city": {
                    "type": "string",
                    "description": "The city to get the temperature for.",
                }
            },
            "required": ["city"]
        }
    }
}

In [25]:
from pydantic import BaseModel, Field

class GetTemperatureArgs(BaseModel):
    city: str = Field(..., description="The city to get the temperature for.")

schema = {
    "type": "function",
    "function": {
        "name": "get_temperature",
        "description": "Get the current temperature in a given city.",
        "parameters": GetTemperatureArgs.model_json_schema()
    }
}


In [32]:
response = client.chat.completions.create(
    messages=[
        {"role": "user", "content": "what is the temperature in San Francisco today?"}
    ],
    tools=[schema],
    tool_choice="auto" # Let the model decide when to call functions
)

In [33]:
print(response.choices[0].message.reasoning_content)

The user is asking for the temperature in San Francisco today. I have access to a function called `get_temperature` that can get the current temperature in a given city. I should call this function with "San Francisco" as the city parameter.


In [36]:
print(response.choices[0].message.tool_calls[0])

ChatCompletionOutputToolCall(function=ChatCompletionOutputFunctionDefinition(arguments='{"city": "San Francisco"}', name='get_temperature', description=None), id='get_temperature:0', type='function', index=0)


In [37]:
import json

class Agent:
    def __init__(self, client: InferenceClient, system: str = "", tools: list = None) -> None:
        self.client = client
        self.system = system
        self.messages: list = []
        self.tools = tools if tools is not None else []
        if self.system:
            self.messages.append({"role": "system", "content": system})

    def __call__(self, message=""):
        if message:
            self.messages.append({"role": "user", "content": message})

        final_assistant_content = self.execute()

        if final_assistant_content:
            self.messages.append({"role": "assistant", "content": final_assistant_content})

        return final_assistant_content

    def execute(self):
        # Keep looping until the model provides a final text response (not tool calls)
        while True:
            completion = self.client.chat.completions.create(
                messages=self.messages,
                tools=self.tools,
                tool_choice="auto" # Allow the model to decide whether to call tools
            )

            response_message = completion.choices[0].message

            if response_message.tool_calls:
                self.messages.append(response_message)

                tool_outputs = []
                for tool_call in response_message.tool_calls:
                    function_name = tool_call.function.name
                    function_args = json.loads(tool_call.function.arguments)

                    # Execute the tool
                    if function_name in globals() and callable(globals()[function_name]):
                        function_to_call = globals()[function_name]
                        executed_output = function_to_call(**function_args)
                        tool_output_content = str(executed_output) # Ensure output is string
                        print(f"Executing tool: {function_name} with args {function_args}, Output: {tool_output_content[:500]}...") # Debug print

                    tool_outputs.append(
                        {
                            "tool_call_id": tool_call.id,
                            "role": "tool",
                            "name": function_name,
                            "content": tool_output_content,
                        }
                    )

                self.messages.extend(tool_outputs)

            else:
                return response_message.content

In [39]:

agent = Agent(
    client=client,
    system="You are a helpful assistant that can answer questions using the provided tools.",
    tools=[schema]
)

response = agent("what is the temperature in san francisco?")
print(response)

Executing tool: get_temperature with args {'city': 'San Francisco'}, Output: 72...
The temperature in San Francisco is **72°F**.


In [38]:
schema

{'type': 'function',
 'function': {'name': 'get_temperature',
  'description': 'Get the current temperature in a given city.',
  'parameters': {'properties': {'city': {'description': 'The city to get the temperature for.',
     'title': 'City',
     'type': 'string'}},
   'required': ['city'],
   'title': 'GetTemperatureArgs',
   'type': 'object'}}}

In [40]:
agent.messages

[{'role': 'system',
  'content': 'You are a helpful assistant that can answer questions using the provided tools.'},
 {'role': 'user', 'content': 'what is the temperature in san francisco?'},
 ChatCompletionOutputMessage(role='assistant', content='', reasoning=None, tool_call_id=None, tool_calls=[ChatCompletionOutputToolCall(function=ChatCompletionOutputFunctionDefinition(arguments='{"city": "San Francisco"}', name='get_temperature', description=None), id='get_temperature:0', type='function', index=0)], reasoning_content='The user is asking for the current temperature in San Francisco. I have access to a get_temperature function that can retrieve the temperature for a given city. Let me call that function with "San Francisco" as the city.'),
 {'tool_call_id': 'get_temperature:0',
  'role': 'tool',
  'name': 'get_temperature',
  'content': '72'},
 {'role': 'assistant',
  'content': 'The temperature in San Francisco is **72°F**.'}]